#Autogen Stock Price Analyst Chatbot:

Project Purpose
The primary objective of this project is to create an investment chatbot by using Autogen. the chatbot specifically designed to assist users in analyzing stock market data, with a focus on individual stock performance.

Architecture and Components
The chatbot is built using the Autogen framework and consists of two main agents:

1. User Proxy Agent (Rattanon): Acts as the interface for the user, receiving commands and input.

2. Assistant Agent (Mr. Analyst): Functions as a financial expert, capable of analyzing stock data and utilizing external tools.

APIs Used
- Alpha Vantage: Used for retrieving daily stock data, including:

    - Date

    - Open Price

    - High Price

    - Low Price

    - Close Price

    - Volume

LLM (Large Language Model):

- Gemini 2.5 Flash: Utilized for the agents' conversational and analytical capabilities.

Agent Capabilities
The agents in this chatbot can perform basic analysis of individual stock data, focusing on:

- Price fluctuation

- Volatility

- Recent trends



Chatbot Workflow
The interaction flow within the chatbot follows these steps:

1. Initial Setup:

Load and configure API Keys for Gemini and Alpha Vantage.

2. Agent Role Assignment:

Rattanon (User Proxy Agent): Configured to receive user input and initiate conversations.

Mr. Analyst (Assistant Agent): Configured as a financial expert with the ability to use the get_stock_data tool to fetch stock information.

The get_stock_data function is registered and made callable by Mr. Analyst, with execution handled by Rattanon.

3. Initiating Conversation:

The user starts a conversation with Mr. Analyst through Rattanon.

Example: User prompts with "get me data on aapl".

    - Mr. Analyst interprets the request and suggests using the get_stock_data tool.

    - The user confirms the tool call by pressing Enter.

    - The get_stock_data function executes, fetching daily stock data from the Alpha Vantage API.

    - The retrieved data is sent back to Mr. Analyst for analysis, which is then presented in Markdown format.

    - The user can type exit to end the conversation.

In [1]:
!pip install pyautogen
!pip install -q -U google-generativeai
!pip install alpha_vantage

import os
import autogen
import pandas as pd
import google.generativeai as genai
from alpha_vantage.timeseries import TimeSeries

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.5 MB/s eta 0:00:00


# Config API

In [2]:
try:
    from google.colab import userdata
    #get Gemini key
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

    if GOOGLE_API_KEY:
        os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
        print("GOOGLE_API_KEY loaded from Secrets.")
    else:
        print("WARNING: 'GOOGLE_API_KEY' secret not found")

    #get alpha vantage key
    ALPHA_VANTAGE_API_KEY = userdata.get('ALPHA_VANTAGE_API_KEY')

    if ALPHA_VANTAGE_API_KEY:
        os.environ["ALPHA_VANTAGE_API_KEY"] = ALPHA_VANTAGE_API_KEY
        print("ALPHA_VANTAGE_API_KEY loaded from Secrets.")
    else:
        print("WARNING: 'ALPHA_VANTAGE_API_KEY' secret not found.")

except ImportError:
    print("google.colab.userdata' not found. Ensure API key is set via other means if not in Colab.")

if not os.environ.get("GOOGLE_API_KEY"):
    raise RuntimeError("CRITICAL ERROR: GOOGLE_API_KEY environment variable is not set. Please ensure it's loaded via Colab Secrets.")

if not os.environ.get("ALPHA_VANTAGE_API_KEY"):
    raise RuntimeError("CRITICAL ERROR: ALPHA_VANTAGE_API_KEY environment variable is not set. Please ensure it's loaded via Colab Secrets.")

genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

#define model
config_list = [
    {
        "model": "gemini-2.5-flash",
        "api_key": os.environ.get("GOOGLE_API_KEY"),
        "api_type": "google"
    }
]

## Config the model for autogen
llm_config = {
    "config_list": config_list,
    "temperature": 0.7
}

print("API Key handling setup complete.")


GOOGLE_API_KEY loaded from Secrets.
ALPHA_VANTAGE_API_KEY loaded from Secrets.
API Key handling setup complete.


#create get_stock_data to retrieve daily stock data.

Description:
This function serves to fetch daily stock price data for a specified stock symbol using the Alpha Vantage API and then tidies it up into a neat Markdown string. So it's super easy for our super smart LLM buddies to read and understand! Think of it as translating complex stock charts into a friendly, readable report.

In [3]:
def get_stock_data(symbol: str, outputsize: str = "compact") -> str:
    try:
        # Initialize TimeSeries object with Alpha Vantage API key
        ts = TimeSeries(key=os.environ.get("ALPHA_VANTAGE_API_KEY"), output_format='pandas')
        # Fetch daily stock data for the given symbol and output size
        data, meta_data = ts.get_daily(symbol=symbol, outputsize=outputsize)
        # Convert DataFrame to Markdown string for LLM readability
        return data.to_markdown()

    # error handling
    except Exception as e:
        error_message = f"Error fetching data from Alpha Vantage: {e}"
        print(error_message) # Print the error for debugging
        return error_message

# Agents creation

we'll be create 2 agents
  1. user_proxy named Rattanon to represent the user in the conversation
  2. analyst named Mr. analyst to be our stock expert, giving analysis based on the stock price data

and also don't forget to give our agent the tool (get_stock_data function) that we declared beforehand, so our agent can do their job properly.

In [4]:
# create user_proxy to represent user
user_proxy = autogen.UserProxyAgent(
    name="Rattanon",
    human_input_mode="ALWAYS", # Always waits for input
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: "terminate" in x.get("content", "").lower(),
    code_execution_config={"work_dir": "coding", "use_docker": False}, # Configuration for code execution; specifies working directory and no Docker use.
    system_message=( # set capability of user_proxy
         "You are the user. You can ask analyst for financial stock analysis. If you need stock data, ask analyst to retrieve it using the 'get_stock_data' tool. Specify the stock symbol and optionally the outputsize (compact or full)."
    )
)

# create analyst agent
analyst = autogen.AssistantAgent(
    name="Mr.analyst",
    llm_config={"config_list": config_list},
    system_message=( # set capability of analyst
         "You are a financial analyst. Provide clear and concise analysis of stock data. When asked for an opinion on a stock, synthesize information from the provided data and your general understanding of market dynamics, industry trends, and company fundamentals. Always include disclaimers about investment risks and note when opinions are based on broader market context rather than just specific data points."
    )
)
# register get_stock_data to analyst
autogen.agentchat.register_function(
    get_stock_data,
    caller=analyst,
    executor=user_proxy,
    name="get_stock_data",
    description="Retrieves daily stock data for a given symbol from Alpha Vantage. Args: symbol (str), outputsize (str, optional, 'compact' or 'full')."
)

# chat initiation! let's chat with our stock expert and ask him about apple and nvdia stocks.



In [6]:
# start the conversation from user prompt the analyst
result = user_proxy.initiate_chat(
    analyst,
    max_turns=30,
)

>hello
Rattanon (to Mr.analyst):

hello

--------------------------------------------------------------------------------
Mr.analyst (to Rattanon):

Hello! I am a financial analyst here to provide you with clear and concise analysis of stock data. How can I help you today?


--------------------------------------------------------------------------------
Replying as Rattanon. Provide feedback to Mr.analyst. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: can you help me analyze aapl?
Rattanon (to Mr.analyst):

can you help me analyze aapl?

--------------------------------------------------------------------------------
Mr.analyst (to Rattanon):


***** Suggested tool call (2467): get_stock_data *****
Arguments: 
{"outputsize": "compact", "symbol": "aapl"}
******************************************************

--------------------------------------------------------------------------------
Replying as Rattanon. Provide feedback to Mr.analyst. Press ente

/usr/local/lib/python3.11/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `Schema` - serialized value may not be as expected [input_value={'type': 'OBJECT', 'prope... 'required': ['symbol']}, input_type=dict])
  return self.__pydantic_serializer__.to_python(
/usr/local/lib/python3.11/dist-packages/autogen/oai/gemini.py:960: UserWarning: Cost calculation is not implemented for model gemini-2.5-flash. Cost will be calculated zero.
  warnings.warn(


Replying as Rattanon. Provide feedback to Mr.analyst. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: can you also analyze nvda as well?
Rattanon (to Mr.analyst):

can you also analyze nvda as well?

--------------------------------------------------------------------------------
Mr.analyst (to Rattanon):


***** Suggested tool call (7687): get_stock_data *****
Arguments: 
{"outputsize": "compact", "symbol": "nvda"}
******************************************************

--------------------------------------------------------------------------------
Replying as Rattanon. Provide feedback to Mr.analyst. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION get_stock_data...
Call ID: 7687
Input arguments: {'outputsize': 'compact', 'symbol': 'nvda'}
Rattanon (to Mr.analyst):

***** Response from calling tool (7687) *****
| date          

/usr/local/lib/python3.11/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `Schema` - serialized value may not be as expected [input_value={'type': 'OBJECT', 'prope... 'required': ['symbol']}, input_type=dict])
  return self.__pydantic_serializer__.to_python(
/usr/local/lib/python3.11/dist-packages/autogen/oai/gemini.py:960: UserWarning: Cost calculation is not implemented for model gemini-2.5-flash. Cost will be calculated zero.
  warnings.warn(


Replying as Rattanon. Provide feedback to Mr.analyst. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: exit

>>>>>>>> TERMINATING RUN (b684575e-38cb-4ecf-b602-c31032adc06e): User requested to end the conversation
